In [70]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
import string
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.models import Model
from sklearn.metrics import precision_recall_fscore_support
import json

from sklearn.metrics import confusion_matrix
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,Dropout , Input , Bidirectional , concatenate, GRU,Conv1D
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
#from tensorflow.keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD, Adam
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
from sklearn.base import BaseEstimator, TransformerMixin
import emoji
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.models import load_model


In [71]:
def load_training_data_to_pandas(filename = 'data/train.jsonl'):
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)

        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        Y.append(data['label'])
 
    
    dfdata = pd.DataFrame({'Tweets': X,'Labels': Y}) 

    dfdata.to_csv(r'data/dataPandas.csv',index=False)

In [3]:
def load_test_data_to_pandas(filename = 'data/test.jsonl'):
    tid = []
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)
        tid.append(data['id'])
        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        
    
    dftestdata = pd.DataFrame({'ID': tid,
                   'Tweets': X})
    
   
    dftestdata.to_csv(r'data/dftestdata.csv',index=False)


    #return X_train,Y_train,X_test,Y_test,maxLen

In [4]:
load_training_data_to_pandas()
load_test_data_to_pandas()

In [5]:
twitterdata = pd.read_csv("data/dataPandas.csv")
twitterdata.isnull().values.any()
twitterdata.shape

(5000, 2)

In [6]:
class TextCounts(BaseEstimator, TransformerMixin):
    
    def count_regex(self, pattern, tweet):
        return len(re.findall(pattern, tweet))
    
    def fit(self, X, y=None, **fit_params):
        # fit method is used when specific operations need to be done on the train data, but not on the test data
        return self
    
    def transform(self, X, **transform_params):
        count_words = X.apply(lambda x: self.count_regex(r'\w+', x)) 
        count_mentions = X.apply(lambda x: self.count_regex(r'@\w+', x))
        count_hashtags = X.apply(lambda x: self.count_regex(r'#\w+', x))
        count_capital_words = X.apply(lambda x: self.count_regex(r'\b[A-Z]{2,}\b', x))
        count_excl_quest_marks = X.apply(lambda x: self.count_regex(r'!|\?', x))
        count_urls = X.apply(lambda x: self.count_regex(r'http.?://[^\s]+[\s]?', x))
        # We will replace the emoji symbols with a description, which makes using a regex for counting easier
        # Moreover, it will result in having more words in the tweet
        count_emojis = X.apply(lambda x: emoji.demojize(x)).apply(lambda x: self.count_regex(r':[a-z_&]+:', x))
        
        df = pd.DataFrame({'count_words': count_words
                           , 'count_mentions': count_mentions
                           , 'count_hashtags': count_hashtags
                           , 'count_capital_words': count_capital_words
                           , 'count_excl_quest_marks': count_excl_quest_marks
                           , 'count_urls': count_urls
                           , 'count_emojis': count_emojis
                          })
        
        return df
    
tc = TextCounts()
twitterdata_eda = tc.fit_transform(twitterdata.Tweets)
#twitterdata_eda['Labels'] = twitterdata.Labels

In [7]:
twitterdata_meta = twitterdata_eda.to_numpy()
twitterdata_meta.shape

(5000, 7)

In [8]:
class CleanText(BaseEstimator, TransformerMixin):
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)    
   
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [9]:
ct = CleanText()
twitterdata_CT = ct.fit_transform(twitterdata.Tweets)
#twitterdata_CT.head()
twitterdata['cTweets'] = twitterdata_CT

In [10]:
X = []
sentences = list(twitterdata['cTweets'])
for sen in sentences:
    #X.append(preprocess_text(sen))
    X.append(sen)

In [11]:
y = twitterdata['Labels']

y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(twitterdata_eda, y, test_size=0.20, random_state=42)

In [13]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 160

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [14]:
embeddings_dictionary = dict()
#glove_file = open('./Data/glove.twitter.27B.100d.txt', encoding="utf8")
glove_file = open('./Data/glove.6B.100d.txt', encoding="utf8")


for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [15]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [80]:
nlp_input1 = Input(shape=(maxlen,)) 
meta_input1 = Input(shape=(7,))
emb1 = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=True)(nlp_input1) 
x = Dense(8, activation="relu")(meta_input1)
nlp_out1 = Bidirectional(GRU(64))(emb1) 
concat1 = concatenate([nlp_out1, x]) 
drop1 = Dropout(0.5)(concat1)
output1 = Dense(1, activation='sigmoid')(drop1) 
model_bgru = Model(inputs=[nlp_input1 , meta_input1], outputs=[output1])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience =1)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model_bgru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_bgru.summary())

Model: "functional_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 160)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 160, 100)     1968800     input_19[0][0]                   
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 7)]          0                                            
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 128)          63744       embedding_9[0][0]                
______________________________________________________________________________________

In [81]:
history = model_bgru.fit([X_train,X_train_meta ], y_train, batch_size=64, epochs=15, verbose=1, validation_data = ([X_test,X_test_meta], y_test), callbacks=[es, mc,rlrp])

Epoch 1/15
63/63 [==============================] - ETA: 0s - loss: 1.4358 - acc: 0.5232
Epoch 00001: val_acc improved from -inf to 0.57300, saving model to best_model.h5
63/63 [==============================] - 8s 131ms/step - loss: 1.4358 - acc: 0.5232 - val_loss: 0.6808 - val_acc: 0.5730
Epoch 2/15
63/63 [==============================] - ETA: 0s - loss: 0.7299 - acc: 0.6370
Epoch 00002: val_acc improved from 0.57300 to 0.68200, saving model to best_model.h5
63/63 [==============================] - 8s 123ms/step - loss: 0.7299 - acc: 0.6370 - val_loss: 0.5841 - val_acc: 0.6820
Epoch 3/15
63/63 [==============================] - ETA: 0s - loss: 0.5054 - acc: 0.7538
Epoch 00003: val_acc improved from 0.68200 to 0.71600, saving model to best_model.h5
63/63 [==============================] - 8s 132ms/step - loss: 0.5054 - acc: 0.7538 - val_loss: 0.5508 - val_acc: 0.7160
Epoch 4/15
63/63 [==============================] - ETA: 0s - loss: 0.4242 - acc: 0.8145
Epoch 00004: val_acc improved

In [82]:

saved_model = load_model('best_model.h5')
score,accuracy = saved_model.evaluate([X_test,X_test_meta], y_test, verbose=1)

32/32 [==============================] - 1s 32ms/step - loss: 0.5653 - acc: 0.7380


In [83]:
print("Test Score:", score)
print("Test Accuracy:", accuracy)

Test Score: 0.5652762651443481
Test Accuracy: 0.7379999756813049


In [84]:

print("Test Score:", score)
print("Test Accuracy:", accuracy)
y_pred = saved_model.predict([X_test,X_test_meta])
y_pred =(y_pred>0.5)
print(len(y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

Test Score: 0.5652762651443481
Test Accuracy: 0.7379999756813049
1000
[[340 160]
 [102 398]]


In [85]:
def myscores(smat): 
    tp = smat[0][0] 
    fp = smat[0][1] 
    fn = smat[1][0] 
    tn = smat[1][1] 
    p = tp/(tp+fp)
    r = tp/(tp+fn)
    f1 = (2*p*r)/(p+r)
    return p,r,f1

print(myscores(cm))

(0.68, 0.7692307692307693, 0.721868365180467)


In [86]:
twittertestdata = pd.read_csv("data/dftestdata.csv")
print(twittertestdata.isnull().values.any())
twittertestdata_CT = ct.fit_transform(twittertestdata.Tweets)
twittertestdata_eda = tc.fit_transform(twittertestdata.Tweets)
twittertestdata_meta = twittertestdata_eda.to_numpy()


twittertestdata['cTweets'] = twittertestdata_CT

False


In [87]:
X_val = []
sentences = list(twittertestdata['cTweets'])
for sen in sentences:
    X_val.append(sen)
    
X_validate = tokenizer.texts_to_sequences(X_val)
#X_valTokens = tokenizer.texts_to_sequences(X_validate)
X_valTokens = pad_sequences(X_validate, padding='post', maxlen=maxlen)

In [88]:
validation = saved_model.predict([X_valTokens,twittertestdata_meta])

In [89]:
twittertestdata['Predict'] = validation

In [90]:
twittertestdata['PLabel'] = np.where(twittertestdata['Predict'] > 0.5, "SARCASM", "NOT_SARCASM")

In [91]:
twittertestdata.head()

,ID,Tweets,cTweets,Predict,PLabel
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...",year old finish read nietzsch ask ayo papa peo...,0.185304,NOT_SARCASM
1,twitter_2,@USER @USER How many verifiable lies has he to...,mani verifi lie told document truth teller sur...,0.835063,SARCASM
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,mayb doc scrub coach mean get hammer gold stan...,0.829247,SARCASM
3,twitter_4,@USER @USER is just a cover up for the real ha...,cover real hate insid left nutshel url hate pl...,0.119860,NOT_SARCASM
4,twitter_5,@USER @USER @USER The irony being that he even...,ironi even ask quit articul consid comment fin...,0.965434,SARCASM


In [92]:
twittertestdata.to_csv('answer_bgru.txt', columns = ["ID" , "PLabel"] , header = False , index = False)

In [29]:
nlp_input1 = Input(shape=(maxlen,)) 
meta_input1 = Input(shape=(7,))
#emb = Embedding(output_dim=embedding_size, input_dim=100, input_length=seq_length)
emb1 = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=True)(nlp_input1) 
#lstm = LSTM(300, dropout=0.3, recurrent_dropout=0.3)(embed)
#nlp_out2 = Bidirectional(LSTM(128))(meta_input1) 
x = Dense(8, activation="relu")(meta_input1)


nlp_out1 = Bidirectional(LSTM(128))(emb1) 
concat1 = concatenate([nlp_out1, x]) 
drop1 = Dropout(0.6)(concat1)
#dens = Dense(1)(drop)

#classifier = Dense(32, activation='relu')(drop) 
output1 = Dense(1, activation='sigmoid')(drop1) 
model_blstm = Model(inputs=[nlp_input1 , meta_input1], outputs=[output1])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience =2)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model_blstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_blstm.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 160)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 160, 100)     1968800     input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256)          234496      embedding_1[0][0]                
_______________________________________________________________________________________

In [30]:
history = model_blstm.fit([X_train,X_train_meta ], y_train, batch_size=64, epochs=15, verbose=1, validation_data = ([X_test,X_test_meta], y_test), callbacks=[es, mc])

Epoch 1/15
63/63 [==============================] - ETA: 0s - loss: 2.3905 - acc: 0.5343
Epoch 00001: val_acc improved from -inf to 0.59500, saving model to best_model.h5
63/63 [==============================] - 20s 321ms/step - loss: 2.3905 - acc: 0.5343 - val_loss: 0.6556 - val_acc: 0.5950
Epoch 2/15
63/63 [==============================] - ETA: 0s - loss: 0.8089 - acc: 0.6453
Epoch 00002: val_acc improved from 0.59500 to 0.69500, saving model to best_model.h5
63/63 [==============================] - 18s 280ms/step - loss: 0.8089 - acc: 0.6453 - val_loss: 0.6507 - val_acc: 0.6950
Epoch 3/15
63/63 [==============================] - ETA: 0s - loss: 0.5401 - acc: 0.7312
Epoch 00003: val_acc did not improve from 0.69500
63/63 [==============================] - 18s 282ms/step - loss: 0.5401 - acc: 0.7312 - val_loss: 0.5945 - val_acc: 0.6810
Epoch 4/15
63/63 [==============================] - ETA: 0s - loss: 0.4866 - acc: 0.7697
Epoch 00004: val_acc improved from 0.69500 to 0.70200, saving

In [31]:
saved_model = load_model('best_model.h5')
score,accuracy = saved_model.evaluate([X_test,X_test_meta], y_test, verbose=1)

32/32 [==============================] - 1s 39ms/step - loss: 0.5864 - acc: 0.7170


In [32]:
print("Test Score:", score)
print("Test Accuracy:", accuracy)
y_pred = saved_model.predict([X_test,X_test_meta])
y_pred =(y_pred>0.6)

cm = confusion_matrix(y_test, y_pred)
print(cm)

Test Score: 0.586414635181427
Test Accuracy: 0.7170000076293945
[[334 166]
 [110 390]]


In [33]:
def myscores(smat): 
    tp = smat[0][0] 
    fp = smat[0][1] 
    fn = smat[1][0] 
    tn = smat[1][1] 
    p = tp/(tp+fp)
    r = tp/(tp+fn)
    f1 = (2*p*r)/(p+r)
    return p,r,f1

print(myscores(cm))

(0.668, 0.7522522522522522, 0.7076271186440679)


In [34]:
validation = saved_model.predict([X_valTokens,twittertestdata_meta])

In [35]:
twittertestdata['Predict'] = validation

In [36]:
twittertestdata['PLabel'] = np.where(twittertestdata['Predict'] > 0.5, "SARCASM", "NOT_SARCASM")

In [37]:
twittertestdata.head()

,ID,Tweets,cTweets,Predict,PLabel
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...",year old finish read nietzsch ask ayo papa peo...,0.542652,SARCASM
1,twitter_2,@USER @USER How many verifiable lies has he to...,mani verifi lie told document truth teller sur...,0.936665,SARCASM
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,mayb doc scrub coach mean get hammer gold stan...,0.776727,SARCASM
3,twitter_4,@USER @USER is just a cover up for the real ha...,cover real hate insid left nutshel url hate pl...,0.297881,NOT_SARCASM
4,twitter_5,@USER @USER @USER The irony being that he even...,ironi even ask quit articul consid comment fin...,0.954085,SARCASM


In [38]:
twittertestdata.to_csv('answer_blstm.txt', columns = ["ID" , "PLabel"] , header = False , index = False)